In [4]:
import numpy as np
from itertools import product

print(list(product(*[[0,1], [1, 2]])))

[(0, 1), (0, 2), (1, 1), (1, 2)]


In [7]:
def explore_params(base_params, search_grid):
    '''
    Generate all parameter combinations and run each using run_multi_threaded

    :param base_params: config from which we want to start exploring
    :param search_grid: dict {key: list of values} of values to test for each specified param
    :return:
    '''

    params_to_vary = list(search_grid.keys())
    n_values_per_param = [len(search_grid[p]) for p in params_to_vary]
    print('Total number of experiments : {}, make sure it is reasonable...'.format(np.prod(n_values_per_param)))
    all_values = list(product(*[search_grid[key] for key in params_to_vary]))
    print('All values : {}'.format(all_values))
    list_of_dicts = []
    for param_tuple in all_values:
        list_of_dicts.append({params_to_vary[param_idx]: param_tuple[param_idx] for param_idx in range(len(params_to_vary))})

    print(list_of_dicts)
    
base = {}
search_grid = {'p1' : [0, 1, 5], 'p2':[3,8]}
explore_params(base, search_grid)

Total number of experiments : 6, make sure it is reasonable...
All values : [(0, 3), (0, 8), (1, 3), (1, 8), (5, 3), (5, 8)]
[{'p1': 0, 'p2': 3}, {'p1': 0, 'p2': 8}, {'p1': 1, 'p2': 3}, {'p1': 1, 'p2': 8}, {'p1': 5, 'p2': 3}, {'p1': 5, 'p2': 8}]


In [23]:
from copy import deepcopy
import tqdm
import hashlib
import json
import os
import numpy as np
from multiprocessing import Pool as ThreadPool
from itertools import product
from copy import deepcopy

def plop(dict, seed):
    print('seed {}'.format(seed), dict)
    
def get_id_for_dict(in_dict):
    # Transform a parameter dict into a 16 digits hash for easier storage
    # Forget n_seeds and n_threads, if there is no param named like that it will have no effect
    dict_filtered = {key: in_dict[key] for key in in_dict.keys() if key not in ['n_threads', 'n_seeds']}
    return hashlib.sha256(json.dumps(dict_filtered, sort_keys=True).encode('utf-8')).hexdigest()[:16]
    
def run_multi_threaded(function, params):
    '''
    Run given function with different seeds.

    :param function: the function to execute with each new set of params; signature (params:dict, seed : int) -> None.
                        First step of that function should be to set the seed.
    :param params: the parameters dict to run in multi-threaded (exploration on this done through "explore_params")
    :return: None
    '''

    # Determine the hash for that particular experiment
    hash = get_id_for_dict(params)
    # Print a message for debugging
    print('Exp with id {} and params {}'.format(hash, params))
    out_dir = 'out/raw/{}'.format(hash)

    # Just in case two params gave exactly the same hash
    # If more than 2 exps with same hash, will fail (but should never happen)
    try:
        os.makedirs(out_dir)
        out_dir += '/'
    except FileExistsError:
        out_dir += '_dup'
        os.makedirs(out_dir)
        out_dir += '/'

    with open(out_dir + 'params', 'w') as outfile:
        json.dump(params, outfile)

    pool = ThreadPool(params['n_threads'])

    _ = pool.starmap(function, zip(
            [params for _ in range(params['n_seeds'])],
            range(params['n_seeds']))
            )

    with open(out_dir + 'exited_naturally', 'w') as outfile:
        outfile.write('True')

def explore_params(function, base_params, search_grid):
    '''
    Generate all parameter combinations and run each using run_multi_threaded

    :param base_params: config from which we want to start exploring
    :param search_grid: dict {key: list of values} of values to test for each specified param
    :return:
    '''

    print('Using base configuration {}'.format(base_params))

    params_to_vary = list(search_grid.keys())
    n_variables = len(params_to_vary)
    n_values_per_param = [len(search_grid[p]) for p in params_to_vary]
    print('Total number of experiments : {}, make sure it is reasonable...'.format(np.prod(n_values_per_param)))
    all_values = list(product(*[search_grid[key] for key in params_to_vary]))

    for param_tuple in tqdm.tqdm(all_values):
        tmp = deepcopy(base_params)
        for i in range(n_variables):
            tmp[params_to_vary[i]] = param_tuple[i]
        print('Using parameters {}'.format(tmp))
        # Now, call multi-threaded simulation for these params (not optimal, we could start new threads as soon as 1
        # is done, but should be reasonable if n_threads divides n_seeds (if not, might have to wait for one thread
        # to do full simulation before starting the next batch of 12...)
        run_multi_threaded(function, tmp)
base = {'n_threads' : 3, 'n_seeds':2, 'p2' : -1}
search_grid = {'p1' : [0, 1, 5], 'p2':[3,8]}
explore_params(plop, base, search_grid)

  0%|          | 0/6 [00:00<?, ?it/s]

Using base configuration {'n_threads': 3, 'n_seeds': 2, 'p2': -1}
Total number of experiments : 6, make sure it is reasonable...
Using parameters {'n_threads': 3, 'n_seeds': 2, 'p2': 3, 'p1': 0}
Exp with id fb53734b34b7ba67 and params {'n_threads': 3, 'n_seeds': 2, 'p2': 3, 'p1': 0}
seed 0 {'n_threads': 3, 'n_seeds': 2, 'p2': 3, 'p1': 0}
seed 1 {'n_threads': 3, 'n_seeds': 2, 'p2': 3, 'p1': 0}


 17%|█▋        | 1/6 [00:00<00:00,  6.14it/s]

Using parameters {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 0}
Exp with id c4032988b5826a97 and params {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 0}
seed 0 {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 0}
seed 1 {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 0}


 33%|███▎      | 2/6 [00:00<00:00,  6.66it/s]

...

 50%|█████     | 3/6 [00:00<00:00,  7.24it/s]

Using parameters {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 1}
Exp with id dd0692ab50419b2d and params {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 1}
seed 0 {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 1}
seed 1 {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 1}
Using parameters {'n_threads': 3, 'n_seeds': 2, 'p2': 3, 'p1': 5}
Exp with id 5bebd48569aa7183 and params {'n_threads': 3, 'n_seeds': 2, 'p2': 3, 'p1': 5}
seed 1 {'n_threads': 3, 'n_seeds': 2, 'p2': 3, 'p1': 5}
seed 0 {'n_threads': 3, 'n_seeds': 2, 'p2': 3, 'p1': 5}


 83%|████████▎ | 5/6 [00:00<00:00,  8.48it/s]

Using parameters {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 5}
Exp with id 3b75e18542c1cb68 and params {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 5}
seed 0 {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 5}
seed 1 {'n_threads': 3, 'n_seeds': 2, 'p2': 8, 'p1': 5}


100%|██████████| 6/6 [00:00<00:00,  8.02it/s]
